In [46]:
import torchtext
from torchtext import data
from torchtext.data import Field
import pandas as pd
import numpy as np
import spacy
import preprocessor as p
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize,wordpunct_tokenize
import string
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [47]:
nlp = spacy.load('en')

In [48]:
p.set_options(p.OPT.URL, p.OPT.EMOJI, p.OPT.MENTION, p.OPT.RESERVED)

In [49]:
stopwords = set(stopwords.words('english'))

In [50]:
stopwords.add("'s")
stopwords.add("...")

In [51]:
# def text_preprocessing(x):
#     x = str(x)
    
#     #cleaning url, @name, emojis, smiley etc
#     text = p.clean(x)
    
#     #punctation removal
#     no_punct = [word for word in word_tokenize(text) if word not in string.punctuation]
    
#     #stopword removal
#     no_sw = set(wrd for wrd in no_punct if wrd not in stopwords)
    
#     return " ".join(w for w in no_sw if w).strip()

In [52]:
def generate_bigrams(x):
    n_grams = set(zip(*[x[i:] for i in range(2)]))
    for n_gram in n_grams:
        x.append(' '.join(n_gram))
    return x

### Creating Dataset using TabularDataset from Torchtext

In [53]:
#TEXT = Field(sequential=True, preprocessing = data.Pipeline(lambda x : text_preprocessing(x)),
#             include_lengths=True)

TEXT = Field(tokenize = 'spacy', preprocessing = generate_bigrams, stop_words=stopwords)

#include_lengths - converts TEXT into a tuple of two elements, 
#one element is the numericalised tensor with pads
#other element is the actual length of our sentence

LABEL = data.LabelField(dtype = torch.float)

tweet = data.TabularDataset(path='train_original.csv', format='csv',
                            fields={
                                'text': ('text', TEXT),
                                'target': ('labels', LABEL)
                                   }
                           )

In [54]:
TEXT.build_vocab(tweet,max_size = 15000,
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(tweet)

In [55]:
#TEXT.build_vocab(tweet, max_size=10000)

In [56]:
#LABEL.build_vocab(tweet)

In [57]:
LABEL.vocab.freqs

Counter({'1': 3271, '0': 4342})

In [58]:
#TEXT.vocab.freqs.pop('') # remove empty string

In [59]:
train, valid = tweet.split(split_ratio=0.8)

### Convert Dataset into Iterator 

In [60]:
device = torch.device('cuda')

### BucketIterator - 

>**Shuffles the data in iterator**

>**It is recommended that for test data we should use just itertor, since we are evaluating the model.**

In [61]:
train_iter, val_iter, test_iter = data.BucketIterator.splits((train, valid, None),batch_sizes=(64, 128, 0),
                                                            sort_key=lambda x: len(x.text), device=device)

In [62]:
type(train_iter.data()[0].text)

list

In [63]:
type(val_iter.data()[99].text)

list

In [64]:
test = pd.read_csv("data/test.csv")

In [65]:
%%time
test = data.TabularDataset(path='data/test.csv', format='csv',
                            fields={
                                'text': ('text', TEXT),
                                }
                           )

CPU times: user 482 ms, sys: 0 ns, total: 482 ms
Wall time: 481 ms


In [66]:
test_iter = data.Iterator(dataset=test, device=device, batch_size=128, sort=False)

## Model Building

In [67]:
class FastText(nn.Module):
    def __init__(self, vocab_size, embedding_dim, output_dim, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        
        self.fc = nn.Linear(embedding_dim, output_dim)
        
    def forward(self, text):
        
        #text = [sent len, batch size]
        
        embedded = self.embedding(text)
                
        #embedded = [sent len, batch size, emb dim]
        
        embedded = embedded.permute(1, 0, 2)
        
        #embedded = [batch size, sent len, emb dim]
        
        pooled = F.avg_pool2d(embedded, (embedded.shape[1], 1)).squeeze(1) 
        
        #pooled = [batch size, embedding_dim]
                
        return self.fc(pooled)

In [68]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
OUTPUT_DIM = 1
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = FastText(INPUT_DIM, EMBEDDING_DIM, OUTPUT_DIM, PAD_IDX)

In [69]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 1,500,301 trainable parameters


In [70]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([15002, 100])


In [71]:
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.3178,  0.8416, -0.9851,  ...,  0.1316,  1.2739,  0.6057],
        [-0.3876, -0.5776,  0.1276,  ..., -0.2854,  0.6830, -2.3014],
        [-0.6610, -0.0730,  0.9238,  ..., -0.2256,  0.8148, -0.4405],
        ...,
        [-1.2171,  0.2461, -0.3521,  ..., -0.4227,  0.0034,  0.9644],
        [ 0.0818, -0.6683, -0.4304,  ...,  0.7354, -1.2078,  0.4332],
        [ 0.1607, -0.3702,  0.7969,  ...,  0.5758, -0.1443,  0.9382]])

In [72]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.6610, -0.0730,  0.9238,  ..., -0.2256,  0.8148, -0.4405],
        ...,
        [-1.2171,  0.2461, -0.3521,  ..., -0.4227,  0.0034,  0.9644],
        [ 0.0818, -0.6683, -0.4304,  ...,  0.7354, -1.2078,  0.4332],
        [ 0.1607, -0.3702,  0.7969,  ...,  0.5758, -0.1443,  0.9382]])


In [73]:
TEXT.vocab.stoi[TEXT.unk_token]

0

In [74]:
print(model.embedding.weight.data.shape)

torch.Size([15002, 100])


In [75]:
optimizer = optim.Adam(model.parameters())

In [76]:
criterion = nn.BCEWithLogitsLoss()

In [77]:
model = model.to(device)
criterion = criterion.to(device)

In [78]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [79]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.labels)
        
        acc = binary_accuracy(predictions, batch.labels)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [80]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.labels)
            
            acc = binary_accuracy(predictions, batch.labels)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)


In [81]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [82]:
N_EPOCHS = 25

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iter, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, val_iter, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 0s
	Train Loss: 0.681 | Train Acc: 58.54%
	 Val. Loss: 0.646 |  Val. Acc: 62.24%
Epoch: 02 | Epoch Time: 0m 0s
	Train Loss: 0.649 | Train Acc: 64.51%
	 Val. Loss: 0.602 |  Val. Acc: 67.86%
Epoch: 03 | Epoch Time: 0m 0s
	Train Loss: 0.610 | Train Acc: 71.39%
	 Val. Loss: 0.552 |  Val. Acc: 74.72%
Epoch: 04 | Epoch Time: 0m 0s
	Train Loss: 0.561 | Train Acc: 77.57%
	 Val. Loss: 0.516 |  Val. Acc: 77.51%
Epoch: 05 | Epoch Time: 0m 0s
	Train Loss: 0.515 | Train Acc: 80.48%
	 Val. Loss: 0.494 |  Val. Acc: 79.24%
Epoch: 06 | Epoch Time: 0m 0s
	Train Loss: 0.474 | Train Acc: 82.96%
	 Val. Loss: 0.486 |  Val. Acc: 79.76%
Epoch: 07 | Epoch Time: 0m 0s
	Train Loss: 0.437 | Train Acc: 84.66%
	 Val. Loss: 0.484 |  Val. Acc: 79.58%
Epoch: 08 | Epoch Time: 0m 0s
	Train Loss: 0.406 | Train Acc: 85.90%
	 Val. Loss: 0.491 |  Val. Acc: 80.76%
Epoch: 09 | Epoch Time: 0m 0s
	Train Loss: 0.382 | Train Acc: 86.74%
	 Val. Loss: 0.497 |  Val. Acc: 79.66%
Epoch: 10 | Epoch Time: 0m 0

In [83]:
test_preds = []

with torch.no_grad():
    for batch in test_iter:
        predictions = model(batch.text).squeeze(1)
        preds = predictions.data.cpu().numpy()
        #preds = preds.data.numpy()
        # the actual outputs of the model are logits, so we need to pass these values to the sigmoid function
        preds = 1 / (1 + np.exp(-preds))
        test_preds.append(preds)

In [84]:
predictions = np.where(np.concatenate(test_preds, axis=0) > 0.5, 1, 0)

In [85]:
test_data = pd.read_csv('data/test.csv')
test_data["target"] = predictions
submission_file = test_data[['id', 'target']]
submission_file.to_csv("fxt_txt_Submission_file.csv", index=False)